# <span style = "color:green;">**ASTR-19 Final Project**</span> (Part 1)

### Display an image from a FITS file

Read in numpy and sep:

In [6]:
import numpy as np
import sep

Read in astropy, matplotlib, and import rcParams (which allows you to style your plots):

In [7]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10, 8]      # set all figure sizes to (10,8)

Read the image into a standard 2d numpy array:

In [8]:
data = fits.open('image.fits')

Now, we need to find the mean and standard deviation of the data. These values can then be used for the color scale of our image to visualize the range of data points, creating an image. The grayscale will range from (mean - std) = black, to (mean + std) = white.

In [9]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation = 'nearest', cmap = 'gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

plt.savefig('fits_image.png', bbox_inches='tight')

TypeError: unsupported operand type(s) for /: 'PrimaryHDU' and 'int'

##### Data must be background subtracted to allow for source detection. When using SEP, background estimation and source detection are two separate steps.
### Background subtraction

Set bkg equal to the measure of the spatially variable background and noise on our image:

In [ ]:
bkg = sep.Background(data)

Using bkg, print the values of the global background level (globalback), and the global background RMS (globalrms):

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Evaluate the **background** as a 2d array, and make it the same size as the original image:

In [ ]:
bkg_image = np.array(bkg)

Now, show the **background**:

In [ ]:
plt.imshow(bkg_image, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar();

plt.savefig('bkg_image.png', bbox_inches='tight')

Evaluate the **background noise** as a 2d array, and make it the same size as the original image:

In [ ]:
bkg_rms = bkg.rms()

Now, show the **background noise**:

In [ ]:
plt.imshow(bkg_rms, interpolation = "nearest", cmap = 'gray', origin = 'lower')
plt.colorbar();

plt.savefig('bkg_rms_image.png', bbox_inches='tight')

Subrtact the background variability from the data:

In [ ]:
data_sub = data - bkg

##### Now that we have subtracted the background, we can run object detection on the background subtracted data.
### Object Detection

Set the detection threshold to be a constant value of 1.5$\sigma$, where $\sigma$ is the global background RMS:

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

How many objects were detected?

In [ ]:
len(objects)

Now, lets use objects['x'] and objects['y'] to find the center points of detected objects, and plot over those coordinates with shapes to highlight their position:

In [ ]:
from matplotlib.patches import Ellipse

First, plot the background subtracted image (like we did before, but using 'data_sub' instead of 'data').
Then, plot an ellipse for each detected object:

In [ ]:
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

for i in range (len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width = 6*objects['a'][i],
                height = 6*objects['b'][i],
                angle = objects['theta'][i] * 180.0 / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig('object_detection.png', bbox_inches='tight')

### Aperture photometry
Lastly, we’ll perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects:

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err = bkg.globalrms, gain = 1)

Show the results for the first 10 objects:

In [ ]:
for i in range(10):
    print('object {:d}: flux = {:f} +/- {:f}'.format(i, flux[i], fluxerr[i]))